![Image](https://substackcdn.com/image/fetch/%24s_%21jnzp%21%2Cf_auto%2Cq_auto%3Agood%2Cfl_progressive%3Asteep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc1eca5d1-0462-4833-ae12-7b645a4e3a20_1828x876.png)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1400/1%2AVIC4SGSBHMkZ9ktYdO5ApA.png)

![Image](https://cdn.prod.website-files.com/640f56f76d313bbe39631bfd/67fe59e0c751ab61c59f32dd_customer%20support%20bot.png)

## Stateful Chatbot with LangGraph — Clean, Step-by-Step Guide


---

## 1. Conceptual Overview

A basic LLM chatbot is *stateless*: each call processes only the current input.
A **stateful chatbot** retains prior messages and uses them as context for future responses.

LangGraph enables this by:

* Defining an explicit **state schema**
* Applying **reducers** to control how state evolves
* Using **checkpointers** to persist state between invocations
* Identifying conversations via a **thread_id**

The workflow here is intentionally minimal: a **single-node sequential graph**.

---

## 2. State Definition

```python
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
```

```python
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
```

### What this does

* `messages` holds the full conversation history.
* `BaseMessage` supports all LangChain message types:

  * `HumanMessage`
  * `AIMessage`
  * `SystemMessage`
  * `ToolMessage`
* `add_messages` is a **LangGraph reducer** that:

  * Appends new messages
  * Prevents overwriting prior history

This is the core mechanism that enables memory accumulation.

---

## 3. Node Logic

```python
from langchain_openai import ChatOpenAI
```

```python
model = ChatOpenAI(model="gpt-4o")
```

```python
def chat_node(state: ChatState):
    response = model.invoke(state["messages"])
    return {"messages": [response]}
```

### Execution behavior

* Receives the current `ChatState`
* Sends the **entire message history** to the LLM
* Returns the new AI message as a list
* The reducer merges it into the existing state

No mutation occurs inside the node; state evolution is declarative.

---

## 4. Graph Construction

```python
from langgraph.graph import StateGraph, START, END
```

```python
builder = StateGraph(ChatState)

builder.add_node("chat_node", chat_node)
builder.add_edge(START, "chat_node")
builder.add_edge("chat_node", END)
```

### Graph topology

```
START ──▶ chat_node ──▶ END
```

* Linear, deterministic execution
* One node processes one conversational turn
* State is finalized at `END`

---

## 5. Persistence via Checkpointing

```python
from langgraph.checkpoint.memory import MemorySaver
```

```python
memory = MemorySaver()
chatbot = builder.compile(checkpointer=memory)
```

### What the checkpointer does

* Automatically saves state at graph termination
* Restores state on the next invocation
* Uses in-memory storage (RAM-only)

Without a checkpointer, the graph would forget all prior messages after each run.

---

## 6. Thread-Scoped Conversation Memory

```python
config = {"configurable": {"thread_id": "user_123"}}
```

### Why `thread_id` matters

* Acts as a **conversation key**
* Allows multiple independent sessions
* Ensures correct state retrieval for each user

Same `thread_id` → same conversation history.

---

## 7. Runtime Execution Loop

```python
from langchain_core.messages import HumanMessage
```

```python
def run_chat():
    print("Chatbot started. Type 'quit' to exit.")
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "by"]:
            break

        initial_state = {
            "messages": [HumanMessage(content=user_input)]
        }

        output = chatbot.invoke(initial_state, config=config)
        print(f"AI: {output['messages'][-1].content}")
```

### Execution flow per turn

1. User input wrapped in `HumanMessage`
2. LangGraph retrieves stored state via `thread_id`
3. New message appended using `add_messages`
4. Full history sent to the LLM
5. AI response appended and persisted

The user experiences continuous memory without manual state handling.

---

## 8. End-to-End Lifecycle Summary

1. **Input arrives** as a `HumanMessage`
2. **State is restored** from memory (by `thread_id`)
3. **Reducer appends** the new message
4. **LLM processes** full conversation history
5. **AI response appended**
6. **Checkpoint saved** at `END`

All persistence is automatic and transparent.

---

## 9. Key Takeaways

* `add_messages` enables safe, append-only message history
* `MemorySaver` provides session persistence
* `thread_id` scopes memory to individual users
* Graph structure remains minimal and deterministic
* No global variables, no manual state mutation



## Key Interview Answers — LangGraph Stateful Chatbot

**What problem does LangGraph solve compared to a basic LLM chatbot?**
LangGraph provides explicit state management and deterministic workflows. It enables persistent, multi-turn conversations by modeling execution as a graph with controlled state transitions and reducers, rather than ad-hoc message passing.

**What is the role of `ChatState` in this design?**
`ChatState` defines the schema for shared state across nodes. It enforces type safety and makes state evolution explicit. In this case, it holds the full conversation history as a list of `BaseMessage`.

**Why use `add_messages` instead of `operator.add`?**
`add_messages` is a LangGraph-specific reducer optimized for message history. It appends new messages without overwriting prior ones and preserves correct message ordering and types.

**How does persistence work in LangGraph?**
Persistence is implemented via a checkpointer. After graph execution reaches `END`, the checkpointer stores the final state. On the next invocation, the state is automatically restored before execution continues.

**What is `MemorySaver` and when would you use it?**
`MemorySaver` is an in-memory checkpointer. It is suitable for local development, demos, or ephemeral sessions. For production, it is typically replaced with a database-backed checkpointer.

**What is the purpose of `thread_id`?**
`thread_id` scopes state to a specific conversation. It allows multiple concurrent users or sessions by mapping each invocation to the correct persisted state.

**How does the LLM receive conversation context?**
The entire message history stored in `ChatState["messages"]` is passed to `model.invoke()`, ensuring responses are conditioned on all prior turns.

**Why return `{"messages": [response]}` from the node?**
Reducers expect incremental updates. Returning the response as a list allows `add_messages` to merge it into the existing state instead of replacing it.

**What guarantees determinism in this workflow?**
A linear graph topology, explicit state schema, pure node functions, and reducer-based state updates ensure predictable execution.

**How would you scale this design?**
By replacing `MemorySaver` with a persistent store, adding routing or tool nodes, and introducing conditional edges while keeping state and reducers explicit.

**What is the biggest architectural benefit of LangGraph here?**
Separation of concerns: conversation memory, execution flow, and model invocation are decoupled, making the system auditable, testable, and extensible.
